# Exercise #1 - Adding More Tools Exercise [Practice]

The research workflow currently performs research with an LLM and writes it to a PDF. To add a little more fun to your research paper, you'll use AI to generate an image of the research subject.

The main purpose of this exercise is to familiarize yourself with the flow and code before we add durability to it.

Fill in the TODO instructions. Go to the `solution` directory if you need help.

In this exercise, you'll:
  - Call actions with your GenAI Application
  - Extract structured information from LLM responses to coordinate between different actions.

### Setup

Before doing the exercise, you need to:

* Create your `.env` file and supply your API key
* Load the environment variables

### Create a `.env` File

Next you'll create a `.env` file to store your API keys.
In the file browser on the left, create a new file and name it `.env`.
Then double click on the `.env` file and add the following line with your API key.

```
LLM_API_KEY = YOUR_API_KEY
LLM_MODEL = "openai/gpt-4o"
```

By default this notebook uses OpenAI's GPT-4o. **To perform image generation, you will need an OpenAI key**

In [ ]:
# Create .env file at the root of the `exercises` directory
with open("../../.env", "w") as fh:
    fh.write("LLM_API_KEY = YOUR_API_KEY\nLLM_MODEL = openai/gpt-4o")  

# Now open the .env file at the root of `exercises` and replace YOUR_API_KEY with your API key.

### Add Your LLM API Key **Before** Running the Following Code Block

In [ ]:
# Load environment variables and configure LLM settings

import os
from dotenv import load_dotenv

load_dotenv(override=True)


# Get LLM_API_KEY environment variable and print it to make sure that your .env file is properly loaded.
LLM_MODEL = os.getenv("LLM_MODEL", "openai/gpt-4o")
LLM_API_KEY = os.getenv("LLM_API_KEY", None)

print("LLM API Key:", LLM_API_KEY[:3] + "*" * (len(LLM_API_KEY) - 3) if LLM_API_KEY else None)

### Define functions

In the content notebook there were two functions defined, `llm_call` and `create_pdf`. Run the codeblocks to define them here.

In [ ]:
# Run this codeblock
from litellm import completion, ModelResponse

# Sends user prompt to an LLM and returns response

def llm_call(prompt: str) -> ModelResponse:
    response = completion(
      model=LLM_MODEL,
      api_key=LLM_API_KEY,
      messages=[{ "content": prompt,"role": "user"}]
    )
    return response

In [ ]:
# Run this codeblock
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image as RLImage
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
import requests
from io import BytesIO

# Converts text content into a PDF
def create_pdf(content: str, filename: str = "research_report.pdf", image_url: str = None):
    doc = SimpleDocTemplate(filename, pagesize=letter)

    styles = getSampleStyleSheet()
    title_style = ParagraphStyle(
        'CustomTitle',
        parent=styles['Heading1'],
        fontSize=24,
        spaceAfter=30,
        alignment=1
    )

    story = []
    title = Paragraph("Research Report", title_style)
    story.append(title)
    story.append(Spacer(1, 20))

    if image_url is not None:
      img_response = requests.get(image_url)
      img_buffer = BytesIO(img_response.content)
      img = RLImage(img_buffer, width=5*inch, height=5*inch)
      story.append(img)
      story.append(Spacer(1, 20))

    paragraphs = content.split('\n\n')
    for para in paragraphs:
        if para.strip():
            p = Paragraph(para.strip(), styles['Normal'])
            story.append(p)
            story.append(Spacer(1, 12))

    doc.build(story)
    return filename

### Part 1 - Getting the Subject from Our Past Prompt

In the workflow from the presentation, you provided a prompt to the LLM to generate research. We now want to **create an image** to add some fun to our PDF that illustrates the topic of the prompt. 

This prompt was likely a complete sentence, with more detail than you will need to create the image. We can use the LLM to get the subject of the original prompt.
For example: "Give me 5 facts about dogs" -> "dogs"

In [ ]:
# Run this codeblock
research_prompt = input("Enter your research topic or question: ")
print(research_prompt)

In [ ]:
# Step 1: Call the `llm_call` function with the  `subject_prompt` parameter passed in
# Step 2: Run this codeblock
subject_prompt = f"What is the main topic of this sentence? Respond with only the topic in a single word or short phrase. No explanation. The sentence is: {research_prompt}"
research_subject = # TODO Call the `llm_call` function with the subject_prompt parameter passed in
print(research_subject["choices"][0]["message"]["content"])

### Part #2 - Generate an Image of the Subject

Now that we have the subect of your prompt, we will generate an image of it by passing it into the `generate_ai_image` as a parameter.

1. Create a prompt for the image generation that includes the subject of the research report
  * Example prompt: f"A cute, natural image of {subject}."
2. Add the missing arguments to the `image_generation` call.
3. In the second code block, call the `generate_ai_image` function with the `research_subject` parameter passed in.

In [ ]:
# Run this codeblock
from litellm import image_generation

def generate_ai_image(subject: str) -> str:

    image_prompt = # TODO: Create a prompt for the image generation that includes the subject passed in as a parameter to this function

    response = image_generation(
        prompt=# TODO: Add the image_prompt you just created
        model="dall-e-3",
        api_key=LLM_API_KEY
    )

    return response

In [ ]:
# Run this codeblock
from IPython.display import Image, display

research_image = # TODO: Call the generate_ai_image function with the research_subject parameter passed in.
research_image_url = research_image["data"][0]["url"]

display(Image(url=research_image_url))

### Part 3 - Updating the workflow

Now that you have an image, let's create a PDF with your research and image. We want to update the workflow to get the topic, make the LLM call, and generate the report.

1. Add a call to `llm_call` to get the topic of your prompt.
  * The `llm_call` function takes in your `research_prompt`.
2. Add a call to `generate_ai_image` to get an image for your research pdf.
  * The `generate_ai_image` function takes in your `research_topic`.

In [ ]:
# Run this codeblock
try:
    # Make the API call
    print("Welcome to the Research Report Generator!")
    research_prompt = input("Enter your research topic or question: ")
    result = # TODO: Call the `llm_call` function. It takes in `research_prompt`.

    # Extract the response content
    response_content = result["choices"][0]["message"]["content"]

    subject_prompt = f"What is the main topic of this sentence? Respond with only the topic in a single word or short phrase. No explanation. The sentence is: {research_prompt}"

    research_topic_llm_response = llm_call(subject_prompt)
    research_topic = research_topic_llm_response["choices"][0]["message"]["content"]

    ai_image = # TODO: Call the `generate_ai_image` function. It takes in `research_topic`.
    ai_image_url = ai_image["data"][0]["url"]

    pdf_filename = create_pdf(response_content, f"{research_topic}.pdf", ai_image_url)
    print(f"SUCCESS! PDF created: {pdf_filename}")
except Exception as e:
    print(f"Error: {e}")

### Download your image by following these steps:
  - Right-click the PDF file in the file explorer
  - Select "Download"
  - Open it locally on your machine

Awesome, you're done with the exercise! But think of the things that could go wrong in this application. What if your image generation API went down? What if the PDF creation fails after you've already paid for the LLM call? How can we make this code more durable? 

We'll find out in the next notebook.

### What's Next?

This workshop introduced you the importance of durability in GenAI applications. Further your learning with these resources:

### Resources

- [A mental model for agentic AI Applications blogpost](https://temporal.io/blog/a-mental-model-for-agentic-ai-applications)
- [From AI hype to durable reality — why agentic flows need distributed-systems discipline blogpost](https://temporal.io/blog/from-ai-hype-to-durable-reality-why-agentic-flows-need-distributed-systems)